In [2]:
!pip list

Package                 Version
----------------------- -------------------
aiohttp                 3.6.2
asn1crypto              1.3.0
async-timeout           3.0.1
attrs                   19.3.0
backcall                0.2.0
bcrypt                  3.1.7
beautifulsoup4          4.9.1
bleach                  3.1.5
brotlipy                0.7.0
cached-property         1.5.1
certifi                 2020.6.20
cffi                    1.14.0
chardet                 3.0.4
cos-python-sdk-v5       1.8.0
coscmd                  1.8.6.16
cryptography            2.5
cycler                  0.10.0
DateTime                4.3
decorator               4.4.2
defusedxml              0.6.0
dicttoxml               1.7.4
distro                  1.5.0
docker                  4.2.1
docker-compose          1.26.0
dockerpty               0.4.1
docopt                  0.6.2
entrypoints             0.3
gensim                  3.0.0
graphviz                0.8.4
idna                    2.10
idna-ssl            

In [2]:
!pip install scipy==1.2.1

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 24.8 MB 3.9 MB/s eta 0:00:01 eta 0:00:05
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.0
    Uninstalling scipy-1.5.0:
      Successfully uninstalled scipy-1.5.0


In [3]:
!pip install gensim==3.0.0
from gensim.models import Word2Vec

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 59.5 MB 54.8 MB/s eta 0:00:01�████▎                       | 15.5 MB 1.2 MB/s eta 0:00:37��████▋                     | 19.7 MB 1.2 MB/s eta 0:00:34/s eta 0:00:52███████████████▎          | 39.7 MB 4.8 MB/s eta 0:00:05████████████████          | 41.1 MB 4.8 MB/s eta 0:00:04�███████████▌       | 45.7 MB 5.2 MB/s eta 0:00:03████████████████████████      | 48.4 MB 5.2 MB/s eta 0:00:03     |████████████████████████████▌   | 53.0 MB 54.8 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 932 kB/s eta 0:00:011
  Created wheel for gensim: filename=gensim-3.0.0-cp36-cp36m-linux_x86_64.whl size=13259938 sha256=bbce9572f5342841ebc298edea6b87738e4b7763127984cd36650f42676e1577
  Stored in directory: /home/tione/.cache/pip/wheels/1d/75/81/5df5f4beeadef71d0828393ad9bea8f370ec5571e8f3cf0606
Successfully built gensim


In [4]:
import pandas as pd
!pip install lightgbm
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from lightgbm.sklearn import LGBMClassifier
from collections import defaultdict
import gc
import time
pd.set_option('display.max_columns', None)

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 2.0 MB 734 kB/s eta 0:00:01


In [7]:
#腾讯云tione专属
import os
import requests
cred_url = os.environ["QCLOUD_CONTAINER_INSTANCE_CREDENTIALS_URL"]
r = requests.get(cred_url)
secretId = r.json()["TmpSecretId"]
secretKey = r.json()["TmpSecretKey"]
token = r.json()["Token"]

In [6]:
#腾讯云tione专属
import os
from qcloud_cos import CosConfig
from qcloud_cos import CosS3Client
from ti.utils import get_temporary_secret_and_token

#### 指定本地文件路径，可根据需要修改。
local_file = "/home/tione/notebook/feed_info3.csv"

#### 用户的存储桶，修改为存放所需数据文件的存储桶，存储桶获取参考腾讯云对象存储
bucket="data-1300056244"

#### 用户的数据，修改为对应的数据文件路径，文件路径获取参考腾讯云对象存储
data_key="wechat_algo_data1/feed_info3.csv"

#### 获取用户临时密钥
secret_id, secret_key, token = get_temporary_secret_and_token()
config = CosConfig(Region=os.environ.get('REGION'), SecretId=secret_id, SecretKey=secret_key, Token=token, Scheme='https')
client = CosS3Client(config)

####  获取文件到本地
response = client.get_object(
    Bucket=bucket,
    Key=data_key,
)
response['Body'].get_stream_to_file(local_file)

In [3]:
## 此份baseline只适用于新手，不适用于有经验的选手 ##
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from lightgbm.sklearn import LGBMClassifier
from collections import defaultdict
import gc
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
pd.set_option('display.max_columns', None)
def reduce_mem(df, cols):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in tqdm(cols):
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    gc.collect()
    return df
## 从官方baseline里面抽出来的评测函数
def uAUC(labels, preds, user_id_list):
    """Calculate user AUC"""
    user_pred = defaultdict(lambda: [])
    user_truth = defaultdict(lambda: [])
    for idx, truth in enumerate(labels):
        user_id = user_id_list[idx]
        pred = preds[idx]
        truth = labels[idx]
        user_pred[user_id].append(pred)
        user_truth[user_id].append(truth)
    user_flag = defaultdict(lambda: False)
    for user_id in set(user_id_list):
        truths = user_truth[user_id]
        flag = False
        # 若全是正样本或全是负样本，则flag为False
        for i in range(len(truths) - 1):
            if truths[i] != truths[i + 1]:
                flag = True
                break
        user_flag[user_id] = flag
    total_auc = 0.0
    size = 0.0
    for user_id in user_flag:
        if user_flag[user_id]:
            auc = roc_auc_score(np.asarray(user_truth[user_id]), np.asarray(user_pred[user_id]))
            total_auc += auc 
            size += 1.0
    user_auc = float(total_auc)/size
    return user_auc
def process_jingyingfanwei(x):
    vals = [i.strip() for i in x[1:-1].split(',')]
    return ' '.join(vals)
def tf_idf(all_data, col, n_components=5):
    Tfidf_vect = TfidfVectorizer()
    tfidf_vec = Tfidf_vect.fit_transform(all_data[col])
    svd_enc = TruncatedSVD(n_components=n_components, n_iter=20, random_state=2020)
    vec_svd = svd_enc.fit_transform(tfidf_vec)
    vec_svd = pd.DataFrame(vec_svd)
    vec_svd.columns = ['SVD_{}_{}'.format(col, i) for i in range(n_components)]
    vec_svd = vec_svd.reset_index(drop= True)
    return vec_svd
y_list = ['read_comment', 'like', 'click_avatar', 'forward', 'favorite', 'comment', 'follow']
max_day = 15
## 读取训练集
train = pd.read_csv("/home/tione/notebook/user_action.csv")
print(train.shape)
for y in y_list:
    print(y, train[y].mean())
## 读取测试集
test = pd.read_csv("/home/tione/notebook/test_a.csv")
test['date_'] = max_day
print(test.shape)
## 合并处理
df = pd.concat([train, test], axis=0, ignore_index=True)
print(df.head(3))
## 读取视频信息表
feed_info = pd.read_csv("/home/tione/notebook/feed_info3.csv")
## 此份baseline只保留这三列
feed_info = feed_info[[
    'feedid', 'authorid', 'videoplayseconds','bgm_song_id','bgm_singer_id','description','ocr','asr'
]]
df = df.merge(feed_info, on='feedid', how='left')
## 视频时长是秒，转换成毫秒，才能与play、stay做运算
df['videoplayseconds'] *= 1000
## 是否观看完视频（其实不用严格按大于关系，也可以按比例，比如观看比例超过0.9就算看完）
df['is_finish'] = (df['play'] >= df['videoplayseconds']).astype('int8')
df['play_times'] = df['play'] / df['videoplayseconds']

play_cols = [
    'is_finish', 'play_times', 'play', 'stay'
]
df['description'].fillna(1111, inplace=True)
df['ocr'].fillna(1111, inplace=True)
df['asr'].fillna(1111, inplace=True)
jingying_svd = tf_idf(df, 'description', n_components=5)
df = df.reset_index(drop= True)
df = pd.concat([df, jingying_svd], axis=1)
jingying_svd = tf_idf(df, 'ocr', n_components=5)
df = df.reset_index(drop= True)
df = pd.concat([df, jingying_svd], axis=1)
jingying_svd = tf_idf(df, 'asr', n_components=5)
df = df.reset_index(drop= True)
df = pd.concat([df, jingying_svd], axis=1)
df.drop(['description', 'ocr','asr'],axis=1,inplace=True)

(7317882, 13)
read_comment 0.03501586934580252
like 0.02580487086290815
click_avatar 0.007533327266004016
forward 0.0038211876059220415
favorite 0.0013424649372591687
comment 0.00040462527272235326
follow 0.0007211102884687126
(421985, 4)
   userid  feedid  date_  device  read_comment  comment  like   play    stay  \
0       8   71474      1       1           0.0      0.0   1.0  500.0  5366.0   
1       8   73916      1       1           0.0      0.0   0.0  250.0  1533.0   
2       8   50282      1       1           0.0      0.0   0.0  750.0  1302.0   

   click_avatar  forward  follow  favorite  
0           0.0      0.0     0.0       0.0  
1           0.0      0.0     0.0       0.0  
2           0.0      0.0     0.0       0.0  


In [5]:
def emb_adjust(df, f1, f2):
	emb_size = 8
	tmp = df.groupby(f1, as_index=False)[f2].agg({'{}_{}_list'.format(f1, f2): list})
	sentences = tmp['{}_{}_list'.format(f1, f2)].values.tolist()
	for i in range(len(sentences)):
		sentences[i] = [str(x) for x in sentences[i]]
	model = Word2Vec(sentences, window=6, min_count=1, sg=0, hs=0, seed=1)

	index_dict = {}
	emb_matrix = []
	for i in tqdm(range(len(sentences))):
		seq = sentences[i]
		vec = []
		for w in seq:
			if w in model.wv.vocab:
				vec.append(model.wv[w])
		if len(vec) > 0:
			emb_matrix.append(np.mean(vec, axis=0))
		else:
			emb_matrix.append([0] * emb_size)
		index_dict[tmp[f1][i]] = i
	emb_matrix = np.array(emb_matrix)
	for i in range(emb_size):
		tmp['{}_of_{}_emb_{}'.format(f1, f2, i)] = emb_matrix[:, i]

	tmp_f2 = df.groupby(f2, as_index=False)[f1].agg({'{}_{}_list'.format(f2, f1): list})
	sentences_f2 = tmp_f2['{}_{}_list'.format(f2, f1)].values.tolist()
	index_dict_f2 = {}
	emb_matrix_f2 = []
	for i in tqdm(range(len(sentences_f2))):
		seq = sentences_f2[i]
		vec = []
		for w in seq:
			vec.append(emb_matrix[index_dict[w]])
		if len(vec) > 0:
			emb_matrix_f2.append(np.mean(vec, axis=0))
		else:
			emb_matrix_f2.append([0] * emb_size)
		index_dict_f2[str(tmp_f2[f2][i])] = i
	emb_matrix_f2 = np.array(emb_matrix_f2)

	emb_matrix_adjust = []
	for seq in tqdm(sentences):
		vec = []
		for w in seq:
			vec.append(emb_matrix_f2[index_dict_f2[w]])
		if len(vec) > 0:
			emb_matrix_adjust.append(np.mean(vec, axis=0))
		else:
			emb_matrix_adjust.append([0] * emb_size)
	emb_matrix_adjust = np.array(emb_matrix_adjust)
	for i in range(emb_size):
		tmp['{}_of_{}_emb_adjust_{}'.format(f1, f2, i)] = emb_matrix_adjust[:, i]

	tmp = tmp.drop('{}_{}_list'.format(f1, f2), axis=1)
	
	word_list = []
	emb_matrix2 = []
	for w in tqdm(model.wv.vocab):
		word_list.append(w)
		emb_matrix2.append(model.wv[w])
	emb_matrix2 = np.array(emb_matrix2)
	tmp2 = pd.DataFrame()
	tmp2[f2] = np.array(word_list).astype('int')
	for i in range(emb_size):
		tmp2['{}_emb_{}'.format(f2, i)] = emb_matrix2[:, i]
	
	return tmp, tmp2
def make_feature(df):
    	# embedding特征
	print('开始构造emb特征')
	emb_cols = [['feedid', 'userid','authorid','bgm_song_id','bgm_singer_id']]
	sort_df = df.sort_values('pt_d').reset_index(drop=True)
	for f1, f2 in emb_cols:
		tmp, tmp2 = emb_adjust(sort_df, f1, f2)
		df = df.merge(tmp, on=f1, how='left').merge(tmp2, on=f2, how='left')

In [6]:
# embedding特征

from tqdm import tqdm

from sklearn.model_selection import StratifiedKFold
from gensim.models import Word2Vec
from multiprocessing import Pool
print('开始构造emb特征')
emb_cols = [['feedid','userid']]
sort_df = df.sort_values('date_').reset_index(drop=True)
for f1, f2 in emb_cols:
	tmp, tmp2 = emb_adjust(sort_df, f1, f2)
	df = df.merge(tmp, on=f1, how='left').merge(tmp2, on=f2, how='left')
print('开始构造emb特征')
emb_cols = [['feedid','authorid']]
sort_df = df.sort_values('date_').reset_index(drop=True)
for f1, f2 in emb_cols:
	tmp, tmp2 = emb_adjust(sort_df, f1, f2)
	df = df.merge(tmp, on=f1, how='left').merge(tmp2, on=f2, how='left')
print('开始构造emb特征')
emb_cols = [['feedid','bgm_song_id']]
sort_df = df.sort_values('date_').reset_index(drop=True)
for f1, f2 in emb_cols:
	tmp, tmp2 = emb_adjust(sort_df, f1, f2)
	df = df.merge(tmp, on=f1, how='left').merge(tmp2, on=f2, how='left')
print('开始构造emb特征')
emb_cols = [['feedid','bgm_singer_id']]
sort_df = df.sort_values('date_').reset_index(drop=True)
for f1, f2 in emb_cols:
	tmp, tmp2 = emb_adjust(sort_df, f1, f2)
	df = df.merge(tmp, on=f1, how='left').merge(tmp2, on=f2, how='left')
print('开始构造emb特征')
emb_cols = [['userid','authorid']]
sort_df = df.sort_values('date_').reset_index(drop=True)
for f1, f2 in emb_cols:
	tmp, tmp2 = emb_adjust(sort_df, f1, f2)
	df = df.merge(tmp, on=f1, how='left').merge(tmp2, on=f2, how='left')
print('开始构造emb特征')
emb_cols = [['userid','bgm_song_id']]
sort_df = df.sort_values('date_').reset_index(drop=True)
for f1, f2 in emb_cols:
	tmp, tmp2 = emb_adjust(sort_df, f1, f2)
	df = df.merge(tmp, on=f1, how='left').merge(tmp2, on=f2, how='left')
print('开始构造emb特征')
emb_cols = [['userid','bgm_singer_id']]
sort_df = df.sort_values('date_').reset_index(drop=True)
for f1, f2 in emb_cols:
	tmp, tmp2 = emb_adjust(sort_df, f1, f2)
	df = df.merge(tmp, on=f1, how='left').merge(tmp2, on=f2, how='left')
print('开始构造emb特征')
emb_cols = [['authorid','bgm_song_id']]
sort_df = df.sort_values('date_').reset_index(drop=True)
for f1, f2 in emb_cols:
	tmp, tmp2 = emb_adjust(sort_df, f1, f2)
	df = df.merge(tmp, on=f1, how='left').merge(tmp2, on=f2, how='left')
print('开始构造emb特征')
emb_cols = [['authorid','bgm_song_id']]
sort_df = df.sort_values('date_').reset_index(drop=True)
for f1, f2 in emb_cols:
	tmp, tmp2 = emb_adjust(sort_df, f1, f2)
	df = df.merge(tmp, on=f1, how='left').merge(tmp2, on=f2, how='left')
print('开始构造emb特征')
emb_cols = [['bgm_song_id','bgm_singer_id']]
sort_df = df.sort_values('date_').reset_index(drop=True)
for f1, f2 in emb_cols:
	tmp, tmp2 = emb_adjust(sort_df, f1, f2)
	df = df.merge(tmp, on=f1, how='left').merge(tmp2, on=f2, how='left')

开始构造emb特征


100%|██████████| 20000/20000 [00:00<00:00, 529771.13it/s]


开始构造emb特征


100%|██████████| 18623/18623 [00:00<00:00, 465361.86it/s]


开始构造emb特征


...

100%|██████████| 23738/23738 [00:00<00:00, 444263.73it/s]


开始构造emb特征


100%|██████████| 23738/23738 [00:00<00:00, 453041.09it/s]


开始构造emb特征


100%|██████████| 16601/16601 [00:00<00:00, 480983.94it/s]


In [8]:
df.drop(['bgm_song_id_emb_0_x','bgm_song_id_emb_1_x','bgm_song_id_emb_2_x','bgm_song_id_emb_3_x','bgm_song_id_emb_4_x','bgm_song_id_emb_5_x','bgm_song_id_emb_6_x','bgm_song_id_emb_7_x','bgm_song_id_emb_0_y','bgm_song_id_emb_1_y','bgm_song_id_emb_2_y','bgm_song_id_emb_3_y','bgm_song_id_emb_4_y','bgm_song_id_emb_5_y','bgm_song_id_emb_6_y','bgm_song_id_emb_7_y'], axis=1, inplace=True) # inplace=True, 直接从内部删除


In [9]:
## 统计历史7天的曝光、转化、视频观看等情况（此处的转化率统计其实就是target encoding）
n_day = 7
for stat_cols in tqdm([
    ['userid'],
    ['feedid'],
    ['authorid'],
    ['bgm_song_id'],
    ['bgm_singer_id'],
    ['userid', 'authorid']
]):
    f = '_'.join(stat_cols)
    stat_df = pd.DataFrame()
    for target_day in range(2, max_day + 1):
        left, right = max(target_day - n_day, 1), target_day - 1
        tmp = df[((df['date_'] >= left) & (df['date_'] <= right))].reset_index(drop=True)
        tmp['date_'] = target_day
        tmp['{}_{}day_count'.format(f, n_day)] = tmp.groupby(stat_cols)['date_'].transform('count')
        g = tmp.groupby(stat_cols)
        tmp['{}_{}day_finish_rate'.format(f, n_day)] = g[play_cols[0]].transform('mean')
        feats = ['{}_{}day_count'.format(f, n_day), '{}_{}day_finish_rate'.format(f, n_day)]
        for x in play_cols[1:]:
            for stat in ['max', 'mean']:
                tmp['{}_{}day_{}_{}'.format(f, n_day, x, stat)] = g[x].transform(stat)
                feats.append('{}_{}day_{}_{}'.format(f, n_day, x, stat))
        for y in y_list[:4]:
            tmp['{}_{}day_{}_sum'.format(f, n_day, y)] = g[y].transform('sum')
            tmp['{}_{}day_{}_mean'.format(f, n_day, y)] = g[y].transform('mean')
            feats.extend(['{}_{}day_{}_sum'.format(f, n_day, y), '{}_{}day_{}_mean'.format(f, n_day, y)])
        tmp = tmp[stat_cols + feats + ['date_']].drop_duplicates(stat_cols + ['date_']).reset_index(drop=True)
        stat_df = pd.concat([stat_df, tmp], axis=0, ignore_index=True)
        del g, tmp
    df = df.merge(stat_df, on=stat_cols + ['date_'], how='left')
    del stat_df
    gc.collect()
## 全局信息统计，包括曝光、偏好等，略有穿越，但问题不大，可以上分，只要注意不要对userid-feedid做组合统计就行
for f in tqdm(['userid', 'feedid', 'authorid']):
    df[f + '_count'] = df[f].map(df[f].value_counts())
for f1, f2 in tqdm([
    ['userid', 'feedid'],
    ['userid', 'authorid']
]):
    df['{}_in_{}_nunique'.format(f1, f2)] = df.groupby(f2)[f1].transform('nunique')
    df['{}_in_{}_nunique'.format(f2, f1)] = df.groupby(f1)[f2].transform('nunique')
for f1, f2 in tqdm([
    ['userid', 'authorid']
]):
    df['{}_{}_count'.format(f1, f2)] = df.groupby([f1, f2])['date_'].transform('count')
    df['{}_in_{}_count_prop'.format(f1, f2)] = df['{}_{}_count'.format(f1, f2)] / (df[f2 + '_count'] + 1)
    df['{}_in_{}_count_prop'.format(f2, f1)] = df['{}_{}_count'.format(f1, f2)] / (df[f1 + '_count'] + 1)
df['videoplayseconds_in_userid_mean'] = df.groupby('userid')['videoplayseconds'].transform('mean')
df['videoplayseconds_in_authorid_mean'] = df.groupby('authorid')['videoplayseconds'].transform('mean')
df['feedid_in_authorid_nunique'] = df.groupby('authorid')['feedid'].transform('nunique')

100%|██████████| 1/1 [00:03<00:00,  3.58s/it]


In [10]:
## 内存够用的不需要做这一步
df = reduce_mem(df, [f for f in df.columns if f not in ['date_'] + play_cols + y_list])
train = df[~df['read_comment'].isna()].reset_index(drop=True)
test = df[df['read_comment'].isna()].reset_index(drop=True)
cols = [f for f in df.columns if f not in ['date_'] + play_cols + y_list]
print(train[cols].shape)
trn_x = train[train['date_'] < 14].reset_index(drop=True)
val_x = b[train['date_'] == 14].reset_index(drop=True)
##################### 线下验证 #####################
uauc_list = []
r_list = []
for y in y_list[:4]:
    print('=========', y, '=========')
    t = time.time()
    clf = LGBMClassifier(
        learning_rate=0.05,
        n_estimators=5000,
        num_leaves=63,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=2021,
        metric='None',
    )
    clf.fit(
        trn_x[cols], trn_x[y],
        eval_set=[(val_x[cols], val_x[y])],
        eval_metric='auc',
        early_stopping_rounds=100,
        verbose=50
    )
    val_x[y + '_score'] = clf.predict_proba(val_x[cols])[:, 1]
    val_uauc = uAUC(val_x[y], val_x[y + '_score'], val_x['userid'])
    uauc_list.append(val_uauc)
    print(val_uauc)
    r_list.append(clf.best_iteration_)
    print('runtime: {}\n'.format(time.time() - t))
weighted_uauc = 0.4 * uauc_list[0] + 0.3 * uauc_list[1] + 0.2 * uauc_list[2] + 0.1 * uauc_list[3]
print(uauc_list)
print(weighted_uauc)

100%|██████████| 339/339 [12:45<00:00,  2.26s/it]


14592.85 Mb, 6215.07 Mb (57.41 %)
(7739867, 339)
========= read_comment =========
Training until validation scores don't improve for 100 rounds
[50]	valid_0's auc: 0.930826
[100]	valid_0's auc: 0.932215
[150]	valid_0's auc: 0.932813
[200]	valid_0's auc: 0.933077
[250]	valid_0's auc: 0.933323
[300]	valid_0's auc: 0.933372
[350]	valid_0's auc: 0.933408
[400]	valid_0's auc: 0.933357
Early stopping, best iteration is:
[337]	valid_0's auc: 0.933418
0.6386136741820659
runtime: 2542.3975777626038

========= like =========
Training until validation scores don't improve for 100 rounds
[50]	valid_0's auc: 0.842226
[100]	valid_0's auc: 0.84699
[150]	valid_0's auc: 0.849085
[200]	valid_0's auc: 0.850459
[250]	valid_0's auc: 0.851215
[300]	valid_0's auc: 0.851602
[350]	valid_0's auc: 0.85192
[400]	valid_0's auc: 0.852014
[450]	valid_0's auc: 0.851941
Early stopping, best iteration is:
[370]	valid_0's auc: 0.852061
0.636579909527538
runtime: 2510.065285682678

========= click_avatar =========
Traini

In [26]:
df["date_"].value_counts()

11    614175
14    609036
10    596886
13    566627
3     543566
12    532038
9     512466
4     504623
8     500120
7     480460
1     478489
6     472726
2     461842
5     444828
15    421985
Name: date_, dtype: int64

In [27]:
test = df[df["date_"].isin([15])]

In [29]:
test.head(5)

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite,authorid,videoplayseconds,bgm_song_id,bgm_singer_id,is_finish,play_times,SVD_description_0,SVD_description_1,SVD_description_2,SVD_description_3,SVD_description_4,SVD_ocr_0,SVD_ocr_1,SVD_ocr_2,SVD_ocr_3,SVD_ocr_4,SVD_asr_0,SVD_asr_1,SVD_asr_2,SVD_asr_3,SVD_asr_4,feedid_of_userid_emb_0,feedid_of_userid_emb_1,feedid_of_userid_emb_2,feedid_of_userid_emb_3,feedid_of_userid_emb_4,feedid_of_userid_emb_5,feedid_of_userid_emb_6,feedid_of_userid_emb_7,feedid_of_userid_emb_adjust_0,feedid_of_userid_emb_adjust_1,feedid_of_userid_emb_adjust_2,feedid_of_userid_emb_adjust_3,feedid_of_userid_emb_adjust_4,feedid_of_userid_emb_adjust_5,feedid_of_userid_emb_adjust_6,feedid_of_userid_emb_adjust_7,userid_emb_0,userid_emb_1,userid_emb_2,userid_emb_3,userid_emb_4,userid_emb_5,userid_emb_6,userid_emb_7,feedid_of_authorid_emb_0,feedid_of_authorid_emb_1,feedid_of_authorid_emb_2,feedid_of_authorid_emb_3,feedid_of_authorid_emb_4,feedid_of_authorid_emb_5,feedid_of_authorid_emb_6,feedid_of_authorid_emb_7,feedid_of_authorid_emb_adjust_0,feedid_of_authorid_emb_adjust_1,feedid_of_authorid_emb_adjust_2,feedid_of_authorid_emb_adjust_3,feedid_of_authorid_emb_adjust_4,feedid_of_authorid_emb_adjust_5,feedid_of_authorid_emb_adjust_6,feedid_of_authorid_emb_adjust_7,authorid_emb_0_x,authorid_emb_1_x,authorid_emb_2_x,authorid_emb_3_x,authorid_emb_4_x,authorid_emb_5_x,authorid_emb_6_x,authorid_emb_7_x,feedid_of_bgm_song_id_emb_0,feedid_of_bgm_song_id_emb_1,feedid_of_bgm_song_id_emb_2,feedid_of_bgm_song_id_emb_3,feedid_of_bgm_song_id_emb_4,feedid_of_bgm_song_id_emb_5,feedid_of_bgm_song_id_emb_6,feedid_of_bgm_song_id_emb_7,feedid_of_bgm_song_id_emb_adjust_0,feedid_of_bgm_song_id_emb_adjust_1,feedid_of_bgm_song_id_emb_adjust_2,feedid_of_bgm_song_id_emb_adjust_3,feedid_of_bgm_song_id_emb_adjust_4,feedid_of_bgm_song_id_emb_adjust_5,feedid_of_bgm_song_id_emb_adjust_6,feedid_of_bgm_song_id_emb_adjust_7,feedid_of_bgm_singer_id_emb_0,feedid_of_bgm_singer_id_emb_1,feedid_of_bgm_singer_id_emb_2,feedid_of_bgm_singer_id_emb_3,feedid_of_bgm_singer_id_emb_4,feedid_of_bgm_singer_id_emb_5,feedid_of_bgm_singer_id_emb_6,feedid_of_bgm_singer_id_emb_7,feedid_of_bgm_singer_id_emb_adjust_0,feedid_of_bgm_singer_id_emb_adjust_1,feedid_of_bgm_singer_id_emb_adjust_2,feedid_of_bgm_singer_id_emb_adjust_3,feedid_of_bgm_singer_id_emb_adjust_4,feedid_of_bgm_singer_id_emb_adjust_5,feedid_of_bgm_singer_id_emb_adjust_6,feedid_of_bgm_singer_id_emb_adjust_7,bgm_singer_id_emb_0_x,bgm_singer_id_emb_1_x,bgm_singer_id_emb_2_x,bgm_singer_id_emb_3_x,bgm_singer_id_emb_4_x,bgm_singer_id_emb_5_x,bgm_singer_id_emb_6_x,bgm_singer_id_emb_7_x,userid_of_authorid_emb_0,userid_of_authorid_emb_1,userid_of_authorid_emb_2,userid_of_authorid_emb_3,userid_of_authorid_emb_4,userid_of_authorid_emb_5,userid_of_authorid_emb_6,userid_of_authorid_emb_7,userid_of_authorid_emb_adjust_0,userid_of_authorid_emb_adjust_1,userid_of_authorid_emb_adjust_2,userid_of_authorid_emb_adjust_3,userid_of_authorid_emb_adjust_4,userid_of_authorid_emb_adjust_5,userid_of_authorid_emb_adjust_6,userid_of_authorid_emb_adjust_7,authorid_emb_0_y,authorid_emb_1_y,authorid_emb_2_y,authorid_emb_3_y,authorid_emb_4_y,authorid_emb_5_y,authorid_emb_6_y,authorid_emb_7_y,userid_of_bgm_song_id_emb_0,userid_of_bgm_song_id_emb_1,userid_of_bgm_song_id_emb_2,userid_of_bgm_song_id_emb_3,userid_of_bgm_song_id_emb_4,userid_of_bgm_song_id_emb_5,userid_of_bgm_song_id_emb_6,userid_of_bgm_song_id_emb_7,userid_of_bgm_song_id_emb_adjust_0,userid_of_bgm_song_id_emb_adjust_1,userid_of_bgm_song_id_emb_adjust_2,userid_of_bgm_song_id_emb_adjust_3,userid_of_bgm_song_id_emb_adjust_4,userid_of_bgm_song_id_emb_adjust_5,userid_of_bgm_song_id_emb_adjust_6,userid_of_bgm_song_id_emb_adjust_7,userid_of_bgm_singer_id_emb_0,userid_of_bgm_singer_id_emb_1,userid_of_bgm_singer_id_emb_2,userid_of_bgm_singer_id_emb_3,userid_of_bgm_singer_id_emb_4,userid_of_bgm_singer_id_emb_5,userid_of_bgm_singer_id_

In [30]:
##################### 全量训练 #####################
r_dict = dict(zip(y_list[:4], r_list))
for y in y_list[:4]:
    print('=========', y, '=========')
    t = time.time()
    clf = LGBMClassifier(
        learning_rate=0.05,
        n_estimators=r_dict[y],
        num_leaves=63,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=2021,
    )
    clf.fit(
        train[cols], train[y],
        eval_set=[(train[cols], train[y])],
        early_stopping_rounds=r_dict[y],
        verbose=100
    )
    test[y] = clf.predict_proba(test[cols])[:, 1]
    print('runtime: {}\n'.format(time.time() - t))
test[['userid', 'feedid'] + y_list[:4]].to_csv(
    'sub_%.6f_%.6f_%.6f_%.6f_%.6f.csv' % (weighted_uauc, uauc_list[0], uauc_list[1], uauc_list[2], uauc_list[3]),
    index=False
)

========= read_comment =========
Training until validation scores don't improve for 337 rounds
[100]	valid_0's binary_logloss: 0.0892038
[200]	valid_0's binary_logloss: 0.0869922
[300]	valid_0's binary_logloss: 0.0855286
Did not meet early stopping. Best iteration is:
[337]	valid_0's binary_logloss: 0.0850853


/opt/conda/envs/mxnet_py3/lib/python3.6/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


runtime: 1589.583387374878

========= like =========
Training until validation scores don't improve for 370 rounds
[100]	valid_0's binary_logloss: 0.0873367
[200]	valid_0's binary_logloss: 0.0852429
[300]	valid_0's binary_logloss: 0.083832
Did not meet early stopping. Best iteration is:
[370]	valid_0's binary_logloss: 0.0830085
runtime: 2085.4347195625305

========= click_avatar =========
Training until validation scores don't improve for 308 rounds
[100]	valid_0's binary_logloss: 0.0333592
[200]	valid_0's binary_logloss: 0.0318326
[300]	valid_0's binary_logloss: 0.0305558
Did not meet early stopping. Best iteration is:
[308]	valid_0's binary_logloss: 0.0304568
runtime: 1690.5749423503876

========= forward =========
Training until validation scores don't improve for 107 rounds
[100]	valid_0's binary_logloss: 0.0176902
Did not meet early stopping. Best iteration is:
[107]	valid_0's binary_logloss: 0.0175675
runtime: 680.621310710907



In [11]:
## 统计历史7天的曝光、转化、视频观看等情况（此处的转化率统计其实就是target encoding）
n_day = 7
for stat_cols in tqdm([
    ['userid'],
    ['feedid'],
    ['authorid'],
    ['bgm_song_id'],
    ['bgm_singer_id'],
    ['userid', 'authorid']
]):
    f = '_'.join(stat_cols)
    stat_df = pd.DataFrame()
    for target_day in range(2, max_day + 1):
        left, right = max(target_day - n_day, 1), target_day - 1
        tmp = df[((df['date_'] >= left) & (df['date_'] <= right))].reset_index(drop=True)
        tmp['date_'] = target_day
        tmp['{}_{}day_count'.format(f, n_day)] = tmp.groupby(stat_cols)['date_'].transform('count')
        g = tmp.groupby(stat_cols)
        tmp['{}_{}day_finish_rate'.format(f, n_day)] = g[play_cols[0]].transform('mean')
        feats = ['{}_{}day_count'.format(f, n_day), '{}_{}day_finish_rate'.format(f, n_day)]
        for x in play_cols[1:]:
            for stat in ['max', 'mean']:
                tmp['{}_{}day_{}_{}'.format(f, n_day, x, stat)] = g[x].transform(stat)
                feats.append('{}_{}day_{}_{}'.format(f, n_day, x, stat))
        for y in y_list[:4]:
            tmp['{}_{}day_{}_sum'.format(f, n_day, y)] = g[y].transform('sum')
            tmp['{}_{}day_{}_mean'.format(f, n_day, y)] = g[y].transform('mean')
            feats.extend(['{}_{}day_{}_sum'.format(f, n_day, y), '{}_{}day_{}_mean'.format(f, n_day, y)])
        tmp = tmp[stat_cols + feats + ['date_']].drop_duplicates(stat_cols + ['date_']).reset_index(drop=True)
        stat_df = pd.concat([stat_df, tmp], axis=0, ignore_index=True)
        del g, tmp
    df = df.merge(stat_df, on=stat_cols + ['date_'], how='left')
    del stat_df
    gc.collect()
## 全局信息统计，包括曝光、偏好等，略有穿越，但问题不大，可以上分，只要注意不要对userid-feedid做组合统计就行
for f in tqdm(['userid', 'feedid', 'authorid']):
    df[f + '_count'] = df[f].map(df[f].value_counts())
for f1, f2 in tqdm([
    ['userid', 'feedid'],
    ['userid', 'authorid']
]):
    df['{}_in_{}_nunique'.format(f1, f2)] = df.groupby(f2)[f1].transform('nunique')
    df['{}_in_{}_nunique'.format(f2, f1)] = df.groupby(f1)[f2].transform('nunique')
for f1, f2 in tqdm([
    ['userid', 'authorid']
]):
    df['{}_{}_count'.format(f1, f2)] = df.groupby([f1, f2])['date_'].transform('count')
    df['{}_in_{}_count_prop'.format(f1, f2)] = df['{}_{}_count'.format(f1, f2)] / (df[f2 + '_count'] + 1)
    df['{}_in_{}_count_prop'.format(f2, f1)] = df['{}_{}_count'.format(f1, f2)] / (df[f1 + '_count'] + 1)
df['videoplayseconds_in_userid_mean'] = df.groupby('userid')['videoplayseconds'].transform('mean')
df['videoplayseconds_in_authorid_mean'] = df.groupby('authorid')['videoplayseconds'].transform('mean')
df['feedid_in_authorid_nunique'] = df.groupby('authorid')['feedid'].transform('nunique')

## 内存够用的不需要做这一步
df = reduce_mem(df, [f for f in df.columns if f not in ['date_'] + play_cols + y_list])
train = df[~df['read_comment'].isna()].reset_index(drop=True)
test = df[df['read_comment'].isna()].reset_index(drop=True)
cols = [f for f in df.columns if f not in ['date_'] + play_cols + y_list]
print(train[cols].shape)
trn_x = train[train['date_'] < 14].reset_index(drop=True)
val_x = train[train['date_'] == 14].reset_index(drop=True)
##################### 线下验证 #####################
uauc_list = []
r_list = []
for y in y_list[:4]:
    print('=========', y, '=========')
    t = time.time()
    clf = LGBMClassifier(
        learning_rate=0.05,
        n_estimators=5000,
        num_leaves=63,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=2021,
        metric='None',
    )
    clf.fit(
        trn_x[cols], trn_x[y],
        eval_set=[(val_x[cols], val_x[y])],
        eval_metric='auc',
        early_stopping_rounds=100,
        verbose=50
    )
    val_x[y + '_score'] = clf.predict_proba(val_x[cols])[:, 1]
    val_uauc = uAUC(val_x[y], val_x[y + '_score'], val_x['userid'])
    uauc_list.append(val_uauc)
    print(val_uauc)
    r_list.append(clf.best_iteration_)
    print('runtime: {}\n'.format(time.time() - t))
weighted_uauc = 0.4 * uauc_list[0] + 0.3 * uauc_list[1] + 0.2 * uauc_list[2] + 0.1 * uauc_list[3]
print(uauc_list)
print(weighted_uauc)
##################### 全量训练 #####################
r_dict = dict(zip(y_list[:4], r_list))
for y in y_list[:4]:
    print('=========', y, '=========')
    t = time.time()
    clf = LGBMClassifier(
        learning_rate=0.05,
        n_estimators=r_dict[y],
        num_leaves=63,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=2021,
    )
    clf.fit(
        train[cols], train[y],
        eval_set=[(train[cols], train[y])],
        early_stopping_rounds=r_dict[y],
        verbose=100
    )
    test[y] = clf.predict_proba(test[cols])[:, 1]
    print('runtime: {}\n'.format(time.time() - t))
test[['userid', 'feedid'] + y_list[:4]].to_csv(
    'sub_%.6f_%.6f_%.6f_%.6f_%.6f.csv' % (weighted_uauc, uauc_list[0], uauc_list[1], uauc_list[2], uauc_list[3]),
    index=False
)

100%|██████████| 339/339 [13:29<00:00,  2.39s/it]


14592.85 Mb, 6215.07 Mb (57.41 %)
(7739867, 339)
========= read_comment =========
Training until validation scores don't improve for 100 rounds
[50]	valid_0's auc: 0.930785
[100]	valid_0's auc: 0.932204
[150]	valid_0's auc: 0.932878
[200]	valid_0's auc: 0.933255
[250]	valid_0's auc: 0.933431
[300]	valid_0's auc: 0.933459
[350]	valid_0's auc: 0.933442
[400]	valid_0's auc: 0.933433
Early stopping, best iteration is:
[316]	valid_0's auc: 0.933482
0.6356223678004224
runtime: 1462.1000020503998

========= like =========
Training until validation scores don't improve for 100 rounds
[50]	valid_0's auc: 0.841168
[100]	valid_0's auc: 0.84626
[150]	valid_0's auc: 0.8484
[200]	valid_0's auc: 0.849466
[250]	valid_0's auc: 0.850138
[300]	valid_0's auc: 0.8507
[350]	valid_0's auc: 0.850904
[400]	valid_0's auc: 0.850964
[450]	valid_0's auc: 0.851064
[500]	valid_0's auc: 0.850994
[550]	valid_0's auc: 0.851007
Early stopping, best iteration is:
[451]	valid_0's auc: 0.851083
0.6328221864788663
runtime: 

ValueError: Input data must be 2 dimensional and non empty.

In [32]:
booster =clf.booster_
importance = clf.feature_importance(importance_type='split')
feature_name = booster.feature_name()
# for (feature_name,importance) in zip(feature_name,importance):
# print (feature_name,importance)
feature_importance = pd.DataFrame({'feature_name':feature_name,'importance':importance} )
feature_importance.to_csv('feature_importance.csv',index=False)

AttributeError: 'LGBMClassifier' object has no attribute 'feature_importance'

In [35]:
from ti import session
ti_session = session.Session()
inputs = ti_session.upload_data(path="sub2.csv", bucket="data-1300056244", key_prefix="contest")

In [37]:
features_list = df.columns.values

<bound method Booster.feature_importance of <lightgbm.basic.Booster object at 0x7f32a2000048>>